In [14]:
%run ../../Utils/yp_utils.py
import re

# Initial setup

In [2]:
paper_pmid = 12482937
paper_name = 'chang_brown_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [7]:
original_data = pd.read_csv('raw_data/MMS Sensitive Strains.txt', header=None, sep='\n')

In [8]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 103 x 1


In [9]:
original_data.head()

,0
0,AAT2* 3  Amino acid metabolism +++
1,RAD1*† 0  DNA repair +
2,ANC1*† 0  Pol II transcription +++
3,RAD5/REV2* 2  DNA repair +++
4,AOR1* 1  Unknown +++


In [18]:
original_data['gene'] = original_data[0].apply(lambda x: re.split(' |\/',x)[0])
original_data['data'] = original_data[0].apply(lambda x: re.split(' ',x)[-1])

In [19]:
original_data.head()

,0,gene,data
0,AAT2* 3  Amino acid metabolism +++,AAT2*,+++
1,RAD1*† 0  DNA repair +,RAD1*†,+
2,ANC1*† 0  Pol II transcription +++,ANC1*†,+++
3,RAD5/REV2* 2  DNA repair +++,RAD5,+++
4,AOR1* 1  Unknown +++,AOR1*,+++


In [20]:
original_data['gene'] = original_data['gene'].astype(str)

In [21]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [24]:
typo_fixes = {'KIM3': 'MMS1', 'SW16': 'SWI6'}
original_data['gene'] = original_data['gene'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)

In [25]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, gene, data, orf]
Index: []


In [27]:
original_data['data'] = original_data['data'].apply(lambda x: -len(x.strip()))

In [28]:
original_data.set_index('orf', inplace=True)

In [29]:
original_data = original_data[['data']].copy()

In [30]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [31]:
original_data = original_data.groupby(original_data.index).mean()

In [32]:
original_data.shape

(103, 1)

# Load & process tested strains

In [33]:
tested = pd.read_excel('raw_data/Old array position-before Dec 8, 05.xlsx', sheet_name='Sheet1')

In [34]:
tested.head()

,Plate Number,Plate X,Plate Y,Systematic Name
0,1,1,1,YDR161W
1,1,1,2,YCR017C
2,1,1,3,YDR162C
3,1,1,4,YCR019W
4,1,1,5,YDR163W


In [35]:
tested['orf'] = tested['Systematic Name'].astype(str)

In [36]:
tested['orf'] = clean_orf(tested['orf'])

In [39]:
tested.loc[tested['orf']=='YPL072WA','orf'] = 'YPL072W-A'

In [40]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [41]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             Plate Number  Plate X  Plate Y Systematic Name    orf
index_input                                                       
4081                   11       11       12           mixed  MIXED
4094                   11       12        1           mixed  MIXED
5250                   14       13        1           mixed  MIXED
5252                   14       13        3           mixed  MIXED


In [42]:
tested = tested.loc[t,:]

In [43]:
tested_orfs = tested['orf'].unique()

In [44]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YDR138W']

In [45]:
tested_orfs = list(tested_orfs) + missing

In [46]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [47]:
data = original_data.copy()

In [48]:
dataset_ids = [65]
datasets = datasets.reindex(index=dataset_ids)

In [49]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [50]:
data.head()

dataset_id,65
data_type,value
orf,
YDR161W,0
YCR017C,0
YDR162C,0
YCR019W,0
YDR163W,0


## Subset to the genes currently in SGD

In [51]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [52]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,65
,data_type,value
gene_id,orf,
2287,YDR161W,0
573,YCR017C,0
2288,YDR162C,0
575,YCR019W,0
2289,YDR163W,0


# Normalize

In [53]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [54]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [55]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id         65       
data_type       value valuez
gene_id orf                 
2287    YDR161W     0    0.0
573     YCR017C     0    0.0
2288    YDR162C     0    0.0
575     YCR019W     0    0.0
2289    YDR163W     0    0.0

# Print out

In [56]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [57]:
from IO.save_data_to_db3 import *

In [58]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 12482937...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.80s/it]

Updating the data_modified_on field...
